In [16]:
import numpy as np
from sklearn import neighbors

In [17]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [18]:
datapath = '/media/sf_VBox_Shared/timeseries/UCR_WaveGesture/'

In [19]:
X_train = np.load(datapath+'X_train.npy')
y_train_binary = np.load(datapath+'y_train_binary.npy')
X_val = np.load(datapath+'X_val.npy')
y_val_binary = np.load(datapath+'y_val_binary.npy')
X_test = np.load(datapath+'X_test.npy')
y_test_binary = np.load(datapath+'y_test_binary.npy')

In [68]:
print(X_train.shape)
print(X_val.shape)

(627, 315, 3)
(269, 315, 3)


In [20]:
def transform_dataset(X):
    num_samples, num_timesteps, num_channels = X.shape
    return X.reshape(num_samples, num_timesteps*num_channels)

In [21]:
n_neighbors = 1

In [22]:
clf = neighbors.KNeighborsClassifier(n_neighbors)

In [23]:
clf.fit(transform_dataset(X_train), y_train_binary)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [24]:
val_predict = clf.predict(transform_dataset(X_val))

In [25]:
val_predict

array([0, 2, 2, 2, 2, 7, 2, 6, 1, 1, 6, 1, 2, 4, 5, 4, 6, 1, 4, 1, 5, 5, 0,
       1, 7, 4, 7, 1, 7, 3, 1, 2, 2, 6, 3, 5, 7, 1, 4, 2, 5, 0, 0, 7, 2, 1,
       3, 2, 6, 4, 0, 4, 1, 5, 0, 3, 3, 6, 1, 0, 4, 5, 3, 1, 6, 3, 0, 6, 7,
       0, 0, 1, 7, 0, 5, 4, 4, 1, 5, 6, 4, 5, 1, 6, 6, 5, 4, 7, 2, 4, 3, 3,
       5, 6, 6, 3, 4, 2, 7, 0, 7, 4, 1, 1, 0, 3, 4, 7, 0, 0, 6, 6, 0, 2, 5,
       3, 5, 2, 0, 2, 7, 0, 1, 4, 4, 5, 2, 5, 2, 5, 4, 7, 3, 5, 2, 7, 0, 0,
       0, 2, 2, 3, 5, 4, 6, 0, 4, 6, 2, 0, 4, 4, 0, 4, 5, 4, 1, 3, 6, 5, 1,
       0, 5, 5, 5, 3, 6, 6, 7, 4, 5, 2, 2, 7, 6, 6, 3, 7, 0, 2, 3, 4, 3, 0,
       7, 5, 6, 4, 4, 2, 5, 4, 2, 0, 5, 1, 6, 2, 0, 3, 0, 6, 7, 7, 6, 4, 3,
       0, 6, 1, 0, 6, 6, 1, 1, 0, 4, 2, 3, 6, 3, 7, 5, 5, 2, 0, 0, 4, 3, 3,
       5, 1, 3, 0, 7, 2, 4, 4, 3, 3, 4, 4, 6, 2, 5, 6, 1, 0, 2, 7, 1, 1, 7,
       4, 3, 2, 4, 6, 6, 4, 4, 5, 2, 0, 4, 1, 5, 5, 1])

In [26]:
from sklearn import metrics
metrics.accuracy_score(val_predict, y_val_binary)

0.93308550185873607

In [76]:
X_trainval = np.append(X_train, X_val, axis=0)
y_trainval = np.append(y_train_binary, y_val_binary, axis=0)
print(X_trainval.shape)

(896, 315, 3)


In [77]:
#On test set
clf2 = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(transform_dataset(X_trainval), y_trainval)
test_predict = clf.predict(transform_dataset(X_test))
metrics.accuracy_score(test_predict, y_test_binary)

0.94807370184254602

## DTW

In [59]:
from dtw import dtw
from fastdtw import fastdtw


In [50]:
dist = lambda a, b: norm(a - b, ord=2)
#print(dist(2,4))
a1 = np.ones((2,1))
a0 = np.zeros((2,1))
print(dist(a0,a1))
dtw_func = lambda x,y: dtw(x, y, dist=dist)[0]
print(dtw_func(a0,a1)

1.41421356237
0.5


In [61]:
fastdtw_func = lambda x,y: fastdtw(x, y, dist=dist)[0]

In [ ]:
# This predicts for all validation samples - but it's very slow!
pred = np.zeros_like(y_val_binary)
num_train = 10 #X_train.shape[0]
for i in range(10): #range(X_val.shape[0]):
    x = X_val[i,:,:]
    distances = [dtw_func(x, X_train[j,:,:]) for j in range(num_train)]
    print(i)
    print(distances)
    minind = np.argmin(distances)
    pred[i,:] = y_train_binary[minind,:]

Let's see how slow this is

In [58]:
%%timeit
dtw_func(X_val[0,:,:], X_train[0,:,:])

1 loop, best of 3: 555 ms per loop


In [62]:
%%timeit
fastdtw_func(X_val[0,:,:], X_train[0,:,:])

10 loops, best of 3: 55.5 ms per loop


## Random forest

In [63]:
from sklearn import ensemble

In [64]:
random_forest = ensemble.RandomForestClassifier(n_estimators=500)

In [65]:
random_forest.fit(transform_dataset(X_train), y_train_binary)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [70]:
val_predict_rf = clf.predict(transform_dataset(X_val))

In [67]:
metrics.accuracy_score(val_predict, y_val_binary)

0.93308550185873607

In [78]:
#On test set
random_forest = ensemble.RandomForestClassifier(n_estimators=500)
random_forest.fit(transform_dataset(X_trainval), y_trainval)
test_predict_rf = clf.predict(transform_dataset(X_test))
metrics.accuracy_score(test_predict_rf, y_test_binary)

0.94807370184254602